In [2]:
import os
import json
import pandas as pd

def collect_data(top_folder, ctx_size):
    """ Collects the minimum test loss and corresponding parameters across all subfolders in the main folder. """
    results = []
    # Traverse through each subfolder in the main folder
    for submain_folder in os.listdir(top_folder):
        main_folder = os.path.join(top_folder, submain_folder)
        for subfolder in os.listdir(main_folder):
            subfolder_path = os.path.join(main_folder, subfolder)
            progress_file = os.path.join(subfolder_path, 'progress.csv')
            params_file = os.path.join(subfolder_path, 'params.json')
            
            # Check if both necessary files exist
            if os.path.exists(progress_file) and os.path.exists(params_file):
                try:
                    # Read progress.csv and find the minimum test loss
                    data = pd.read_csv(progress_file)
                    data.fillna(0, inplace=True)
                    # Read params.json
                    with open(params_file, 'r') as file:
                        params = json.load(file)
                        if params['context'] != ctx_size:
                            continue
                        # Collect required params and the corresponding test loss
                        result = {}
                        if 'master_neurons' in params:
                            result['master_neurons'] = params.get('master_neurons')
                        if 'context_store' in params:
                            result['store_embedding_neurons'] = params.get('context_store')
                        result['learning_rate'] = params.get('learning_rate')

                        result['best_dev_loss'] = data['dev_loss'].min()
                        result['test_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['test_loss'].iloc[0]
                        result['train_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['train_loss'].iloc[0]
                        result['best_test_loss'] = data['test_loss'].min()
                        result['best_train_loss'] = data['train_loss'].min()
                        results.append(result)
                    # if 'test_loss' in data.columns:
                    #     min_loss = data['test_loss'].min()
                    #     # Read params.json
                    #     with open(params_file, 'r') as file:
                    #         params = json.load(file)
                    #         context_size = params.get('context_size')
                    #         # Collect required params and the corresponding test loss
                    #         result = {
                    #             'learning_rate': params.get('learning_rate'),
                    #             'best_test_loss': min_loss
                    #         }
                    #         results.append(result)
                except Exception as e:
                    print(f"Error processing files in {subfolder_path}: {e}")

    return results

def create_results_table(main_folder, ctx_size):
    """ Creates a table of the minimum test losses for each combination of learning_rate, context_size, and samples. """
    data = collect_data(main_folder, ctx_size)
    if data:
        # Create DataFrame from collected data
        df = pd.DataFrame(data)
        # Group by the parameters and find the row with the minimum test_loss
        sorted_result_df = df.sort_values(by='best_dev_loss', ascending=True)
        # print(f'ctx_size : {ctx_size}')
        # print(sorted_result_df.head(10).to_string(index=False))
    else:
        print("No data collected. Check the contents of your directories.")
    return sorted_result_df


In [5]:
import pandas as pd

def make_the_result_table(paths):
    context_sizes = [0, 1, 256]

    # Placeholder for results
    results = []

    # Iterate over each path and context size, call the function, and store the top row
    for num_stores, path in paths.items():
        for context_size in context_sizes:
            df = create_results_table(path, context_size)
            top_row = df.iloc[0]  # Get the top row
            # Extract necessary columns and add custom columns
            result_row = {
                "# of stores": num_stores,
                "context size": context_size,
                "Learning Rate": top_row['learning_rate'],
                "Train Loss": top_row['train_loss(at best_dev)'],
                "Dev Loss": top_row['best_dev_loss'],
                "Test Loss": top_row['test_loss(at best_dev)']
            }
            results.append(result_row)

    # Combine all top rows into a single DataFrame
    return pd.DataFrame(results)


In [6]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/50"
}

In [7]:
print(make_the_result_table(paths).to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3             1         0.0100    5.613812  5.608477   5.609850
           3           256         0.0100    5.616237  5.611364   5.612627
           5             1         0.0100    5.256996  5.235224   5.247245
           5           256         0.0001    5.249202  5.237397   5.249729
          10             1         0.0100    5.712263  5.755829   5.720371
          10           256         0.0100    5.731197  5.761904   5.729029
          20             1         0.0010    5.850531  5.839253   5.818287
          20           256         0.0100    5.866663  5.854931   5.832909
          30             1         0.0010    5.548159  5.556480   5.550539
          30           256         0.0100    5.569959  5.574575   5.569556
          50             1         0.0001    6.759303  6.723869   6.789614
          50           256         0.0010    5.389271  5.366309   5.386096
